In [ ]:
import pandas as pd
from sklearn.multioutput import ClassifierChain
from sklearn.datasets import make_multilabel_classification
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split

### CC model

In [ ]:
L_list = [
    ['is_FE','is_LPL','is_LM'],
    ['is_FE','is_LM','is_LPL'],
    ['is_LPL','is_FE','is_LM'],
    ['is_LPL','is_LM','is_FE'],
    ['is_LM','is_FE','is_LPL'],
    ['is_LM','is_LPL','is_FE'],
]
L_list

In [ ]:
L_order = np.array([])
new_y = pd.DataFrame()

for i in L_list:
    L_order = np.append(L_order,i).reshape(-1,3)
    new_y = pd.concat([y.loc[:,i[0]],y.loc[:,i[1]],y.loc[:,i[2]]],axis=1)
    x_train,x_test,y_train,y_test = train_test_split(x,new_y,train_size=0.7)
    
    base_c_list = [RandomForestClassifier(n_estimators=3,max_features ='auto' ,max_depth = 3, max_leaf_nodes=11,random_state=i) for i in range(3)]
    cc_model = ClassifierChain(base_c_list)
    
    for cc in cc_model.base_estimator:
        cc.fit(x_train,y_train)
        
    y_pre = np.array([cc.predict(x_test) for cc in cc_model.base_estimator])
    y_pre = y_pre.mean(axis=0)

### ECC model

In [ ]:
Necc = 10
Nmlcc = 6
base_estimator = [RandomForestClassifier(n_estimators=3, max_features ='auto' ,max_depth = 3, max_leaf_nodes=11,random_state=n) for n in range(3)]
Ecc = [ClassifierChain(base_estimator,random_state=k) for k in range(Nmlcc)]

In [ ]:
for i in range(Necc):
    Rmlcc,Lchain = np.array([]),np.array([])
    for o,ecc in enumerate(Ecc):
        for label in L_list:
            new_y = pd.concat([y.loc[:,label[0]],y.loc[:,label[1]],y.loc[:,label[2]]],axis=1)
            x_train,x_test,y_train,y_test = train_test_split(x,new_y,train_size=0.7)

            for h in ecc.base_estimator:
                h.fit(x_train,y_train)
            y_pre = np.array([e.predict(x_test) for e in ecc.base_estimator])
            y_pre = y_pre.mean(axis=0)